# Data Centre Modelling
## Ryan Jenkinson

### Imports

In [1]:
import numpy as np
import pandas as pd
import pypsa
from dataclasses import dataclass

In [2]:
np.random.seed(42)  # for reproducibility

### Constants

In [3]:
@dataclass
class Costs:
    capital_cost: float
    marginal_cost: float

### Cost Assumptions

In [4]:
## TODO: Sensitivity analysis for costs

## Wind costs
wind_capex_per_mw = 2370000  # £ per MW
wind_lifetime_years = 27
wind_discount_rate = 0.07
annuitised_wind_cost = (wind_capex_per_mw * wind_discount_rate) / (
    1 - (1 + wind_discount_rate) ** -wind_lifetime_years
)
wind_fixed_opex = 76000  # £ per MW per year (fixed O&M)
WIND_COSTS = Costs(
    capital_cost=annuitised_wind_cost + wind_fixed_opex,  # £ per MW per year
    marginal_cost=0.01  # £ per MWh (Variable O&M, assumed small)
)

## Gas costs
gas_capex_per_mw = 631000  # £ per MW
gas_lifetime_years = 25
gas_discount_rate = 0.07
annuitised_gas_cost = (gas_capex_per_mw * gas_discount_rate) / (
    1 - (1 + gas_discount_rate) ** -gas_lifetime_years
)
gas_fixed_opex = 12600  # £ per MW per year (fixed O&M)
gas_fuel_price_mwh = 60  # £ per MWh_fuel (assumed fuel price)
gas_efficiency = 0.34  # Efficiency of gas turbine (MWh_elec / MWh_fuel)
gas_variable_operation_and_management_mwh = (
    7.05  # £ per MWh_elec (non-fuel variable O&M)
)
gas_marginal_cost_fuel = gas_fuel_price_mwh / gas_efficiency  # £ per MWh_elec
GAS_COSTS = Costs(
    capital_cost=annuitised_gas_cost + gas_fixed_opex,  # £ per MW per year
    marginal_cost=gas_marginal_cost_fuel + gas_variable_operation_and_management_mwh  # £ per MWh_elec
)

## BESS (Battery Energy Storage System) costs
bess_energy_capex_per_mwh = 225000  # £ per MWh
bess_power_capex_per_mw = 130000  # £ per MW
bess_lifetime_years = 15
bess_discount_rate = 0.07

annuitised_bess_energy_cost = (bess_energy_capex_per_mwh * bess_discount_rate) / (
    1 - (1 + bess_discount_rate) ** -bess_lifetime_years
)
bess_capital_cost_energy = (
    annuitised_bess_energy_cost  # Cost for energy capacity (Store)
)

annuitised_bess_power_cost = (bess_power_capex_per_mw * bess_discount_rate) / (
    1 - (1 + bess_discount_rate) ** -bess_lifetime_years
)
bess_fixed_opex_per_mw_power = 20000  # Fixed OPEX for power conversion part
bess_capital_cost_power = (
    annuitised_bess_power_cost + bess_fixed_opex_per_mw_power
)  # Cost for power capacity (Links)

bess_round_trip_efficiency = 0.87
standing_loss_per_hour = 0.001
bess_marginal_cost_links = 0.001  # Marginal cost for charging/discharging
BESS_COSTS = Costs(
    capital_cost=bess_capital_cost_energy,  # £ per MWh per year
    marginal_cost=0
)
BESS_LINK_COSTS = Costs(
    capital_cost=bess_capital_cost_power,  # £ per MW per year
    marginal_cost=bess_marginal_cost_links  # £ per MWh_elec
)

In [5]:
# TODO: Put everything relative to this
DATA_CENTRE_LOAD_MW = 115 # MW, constant demand including PUE

In [6]:
# Define simulation period: a full year, hourly
snapshots = pd.date_range(start="2023-01-01 00:00", end="2023-12-31 23:00", freq="h")
n = pypsa.Network(snapshots=snapshots)

# --- DEFINE CARRIERS ---
n.add("Carrier", "AC")  # For AC buses
n.add("Carrier", "gas", co2_emissions=0.185)  # tCO2/MWh_th, typical for natural gas
n.add("Carrier", "offshore_wind")
n.add("Carrier", "battery")  # Carrier for battery components

# --- DEFINE BUSES ---
n.add("Bus", "electricity_bus", carrier="AC")
n.add("Bus", "bess_connection_bus", carrier="AC")

# --- DEFINE LOAD ---
# Load (Data Centre): The data centre is modelled as a load with a constant power demand.
constant_demand_profile = pd.Series(DATA_CENTRE_LOAD_MW, index=n.snapshots)
n.add("Load", "data_centre_load", bus="electricity_bus", p_set=constant_demand_profile)


# --- DEFINE GENERATORS ---
# Generator (Offshore Wind Farm): The offshore wind farm's capacity is extendable.
# TODO: User realistic / historical wind profile

wind_availability_pu = pd.Series(
    np.random.rand(len(n.snapshots)) * 0.6 + 0.2, index=n.snapshots
)
wind_capacity_factor = 0.55  # Target capacity factor for the wind farm
wind_availability_pu = wind_capacity_factor * (wind_availability_pu / wind_availability_pu.mean())
wind_availability_pu = wind_availability_pu.clip(0, 1)

n.add(
    "Generator",
    "offshore_wind_farm",
    bus="electricity_bus",
    carrier="offshore_wind",
    p_nom_extendable=True, # Allow capacity to be extended. To cover what we need to for the data centre load
    capital_cost=WIND_COSTS.capital_cost,
    marginal_cost=WIND_COSTS.marginal_cost,
    p_max_pu=wind_availability_pu,
)


# Generator (Gas Turbine Backup): The gas turbine provides backup, and its capacity is also extendable.

n.add(
    "Generator",
    "gas_turbine",
    bus="electricity_bus",
    carrier="gas",
    p_nom_extendable=True, # Allow capacity to be extended to cover the data centre load
    committable=False,  # Cannot be both committable and extendable for LOPF capacity optimization. TODO: Check if this is correct
    capital_cost=GAS_COSTS.capital_cost,
    marginal_cost=GAS_COSTS.marginal_cost,
)


# --- DEFINE STORAGE (BESS) ---
# Store component representing the energy capacity of the BESS, and each of the links for charging and discharging.
n.add(
    "Store",
    "bess_storage",
    bus="bess_connection_bus",
    carrier="battery",
    e_nom_extendable=True,
    e_cyclic=True,
    standing_loss=standing_loss_per_hour,
    capital_cost=BESS_COSTS.capital_cost,
)
n.add(
    "Link",
    "bess_charger",
    bus0="electricity_bus",
    bus1="bess_connection_bus",
    carrier="battery",
    p_nom_extendable=True,
    capital_cost=BESS_LINK_COSTS.capital_cost,
    efficiency=bess_round_trip_efficiency**0.5,
    marginal_cost=BESS_LINK_COSTS.marginal_cost,
)
n.add(
    "Link",
    "bess_discharger",
    bus0="bess_connection_bus",
    bus1="electricity_bus",
    carrier="battery",
    p_nom_extendable=True,
    capital_cost=0,  # Assuming power CAPEX is fully on charger link, or split if preferred
    efficiency=bess_round_trip_efficiency**0.5,
    marginal_cost=BESS_LINK_COSTS.marginal_cost,
)

# --- CONSISTENCY CHECK & SOLVE ---

# Perform consistency check
try:
    n.consistency_check()
    print("Network consistency check passed before solving.")
except Exception as e:
    print(f"Consistency check error before solving: {e}")

Network consistency check passed before solving.


In [8]:
# Ensure you have a solver installed (e.g., cbc, glpk, gurobi)
solver_name = "cbc"  # or 'glpk', 'gurobi', etc.

def solve_model(n: pypsa.Network, solver_name: str, print_results: bool = True):
    """Solve the PyPSA network model using the specified solver.
    
    Parameters:
    - n: PyPSA Network object
    - solver_name: Name of the solver to use (e.g., 'cbc', 'glpk')
    - print_results: Whether to print the results after solving
    """
    try:
        # Solve the model using the optimize interface, common with linopy backend
        # The `solve_model` method handles the optimization.
        n.optimize(solver_name=solver_name)
        if not print_results:
            return n
        print("Model solved successfully.")

        # --- DISPLAY RESULTS ---
        print("\n--- Optimization Results ---")
        # Objective value is typically stored in n.model.objective after solving with n.optimize
        obj_value = n.model.objective.value
        print(f"Objective Value: {obj_value:.2f} £ (Total Cost)")

        print("\nOptimal Capacities:")
        # Check if components exist in results before trying to access them
        # Optimal capacities are in p_nom_opt (for Generators, Links) and e_nom_opt (for Stores)
        if (
            "offshore_wind_farm" in n.generators.index
            and "p_nom_opt" in n.generators.columns
        ):
            print(
                f"  Offshore Wind Farm (p_nom_opt): {n.generators.loc['offshore_wind_farm', 'p_nom_opt']:.2f} MW"
            )
        if "gas_turbine" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  Gas Turbine (p_nom_opt): {n.generators.loc['gas_turbine', 'p_nom_opt']:.2f} MW"
            )
        if "bess_storage" in n.stores.index and "e_nom_opt" in n.stores.columns:
            print(
                f"  BESS Storage Energy Capacity (e_nom_opt): {n.stores.loc['bess_storage', 'e_nom_opt']:.2f} MWh"
            )
        if "bess_charger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Charger Power Capacity (p_nom_opt): {n.links.loc['bess_charger', 'p_nom_opt']:.2f} MW"
            )
        if "bess_discharger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Discharger Power Capacity (p_nom_opt): {n.links.loc['bess_discharger', 'p_nom_opt']:.2f} MW"
            )

    except Exception as e:
        print(f"An error occurred during model solution or results processing: {e}")
        print("Please ensure you have a compatible solver installed and in your PATH.")
        print("Common solvers: cbc, glpk. You might need to install them.")
        print(
            "Also ensure Linopy (PyPSA's modeling backend) is installed and functioning correctly."
        )
    return n

In [9]:
n = solve_model(n, solver_name="highs", print_results=True)

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 218.28it/s]
INFO:linopy.io: Writing time: 0.25s


Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-lufzvubb has 113885 rows; 52565 cols; 219005 nonzeros
Coefficient ranges:
  Matrix [2e-01, 1e+00]
  Cost   [1e-03, 3e+05]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+02]
Presolving model
61320 rows, 52564 cols, 157680 nonzeros  0s
52560 rows, 43804 cols, 140160 nonzeros  0s
Dependent equations search running on 17520 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
52560 rows, 43804 cols, 140160 nonzeros  0s
Presolve : Reductions: rows 52560(-61325); columns 43804(-8761); elements 140160(-78845)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 8760(1.0074e+06) 0s
      22983     3.1570122290e+06 Pr: 8760(1.83811e+11) 5s
      30016     6.4900362237e+07 Pr: 19737(3.11794e+07) 10s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 52565 primals, 113885 duals
Objective: 7.20e+07
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


      39945     7.1981744809e+07 Pr: 0(0) 12s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-lufzvubb
Model status        : Optimal
Simplex   iterations: 39945
Objective value     :  7.1981744809e+07
Relative P-D gap    :  6.6037165860e-14
HiGHS run time      :         12.29
Writing the solution to /private/var/folders/r9/dprdmb3d7gs81f5ggtwv0lyw0000gn/T/linopy-solve-8etws40g.sol
Model solved successfully.

--- Optimization Results ---
Objective Value: 71981744.81 £ (Total Cost)

Optimal Capacities:
  Offshore Wind Farm (p_nom_opt): 220.34 MW
  Gas Turbine (p_nom_opt): 20.65 MW
  BESS Storage Energy Capacity (e_nom_opt): 196.71 MWh
  BESS Charger Power Capacity (p_nom_opt): 62.01 MW
  BESS Discharger Power Capacity (p_nom_opt): 70.94 MW


In [10]:
import plotly.express as px

# Get the wind power at each timestep in the problem
generator_power = n.generators_t.p
data_centre_power = n.loads_t.p["data_centre_load"]
# battery_power = n.links_t.p["bess_charger"] - n.links_t.p["bess_discharger"]
fig = px.line(
    data_frame=generator_power,
    x=generator_power.index,
    y=generator_power.columns,
    title="Generator Power Output Over Time",
    labels={"value": "Power (MW)", "variable": "Generator"},
    template="plotly_white",
)
fig.show()

In [11]:
total_gas_mwh_usage = n.generators_t.p["gas_turbine"].sum()
total_data_centre_mwh_usage = n.loads_t.p["data_centre_load"].sum()
print(
    f"Ratio of data centre load covered by gas turbine: {total_gas_mwh_usage / total_data_centre_mwh_usage:.2%}"
)

Ratio of data centre load covered by gas turbine: 1.78%


In [12]:
n.model.objective.value

71981744.80938652

## From Gemini (Nuclear PPA prices)
PPA Market Dynamics: The general PPA market in the UK has experienced volatility. While specific nuclear PPA prices for data centres are not public, the strike price for Hinkley Point C's Contract for Difference (CfD) is £92.50 per MWh (in 2012 prices), which serves as a long-term, inflation-indexed reference for new large-scale nuclear, albeit a subsidised rate. Market analysts have noted that premiums in the UK PPA market can add approximately £10/MWh to baseline PPA costs for corporate buyers seeking specific attributes like 24/7 carbon-free power.

In [ ]:
cost_of_running_microgrid_per_year = n.model.objective.value
HOURS_PER_YEAR = 8760  # Total hours in a year
CARBON_FREE_NUCLEAR_PPA_PREMIUM = 10 # £/MWh, assumed premium for carbon-free nuclear PPA
NUCLEAR_PPA_COST_PER_MWH = 133 # 92.5 £2012/MWh in todays money
cost_of_running_with_nuclear_ppa_per_year = DATA_CENTRE_LOAD_MW * HOURS_PER_YEAR * (NUCLEAR_PPA_COST_PER_MWH + CARBON_FREE_NUCLEAR_PPA_PREMIUM)  # £ per year, assuming a PPA of £60/MWh

print(f"Cost of running the microgrid per year: £{cost_of_running_microgrid_per_year:,.2f}")
print(f"Cost of running with a nuclear PPA per year: £{cost_of_running_with_nuclear_ppa_per_year:,.2f}")
# Calculate the percentage difference in costs
cost_difference_percentage = (
    (cost_of_running_microgrid_per_year - cost_of_running_with_nuclear_ppa_per_year)
    / cost_of_running_with_nuclear_ppa_per_year
) * 100
print(f"Percentage difference in costs: {cost_difference_percentage:.2f}%")

Cost of running the microgrid per year: £72,350,526.16
Cost of running with a nuclear PPA per year: £144,058,200.00
Percentage difference in costs: -49.78%


# Plan for the future
1. Initialise data centre load as a constant
2. Loop through wind farms with capacities at different multiple factors of the data centre.
3. Assume constant things like capex cost and capacity factor for wind. We can do a sensitivity analysis of these later.
4. v1: Assume no battery? v2: Assume battery as a MWh of the multiple of the data centre load too.
5. Solve the model to get the least cost way of operating it with gas + battery in each setup. Calculate the % of the total data centre load that was met by gas. Crude calcualtion = MWh of gas / MWh of data centre throughout the year?


## Some options for analysis
* GIVEN a constant data centre load and GIVEN some historical wind profile. Q: How big should the battery and wind farm be priortional to the data centre load to cost-optimally run the system? How much does it cost to run that system? How would that compare to covering the data centre load via a nuclear PPA?
* Alternative route: Given different configurations of wind sizes, but allowing gas to be as much as is needed to cover the data centre load, how much does it cost to run each of those systems? Compared to nuclear PPA?